In [1]:
from dotenv import load_dotenv
load_dotenv("../.env")

True

In [2]:
import fiftyone as fo
import fiftyone.zoo as foz
import fiftyone.brain as fob

In [3]:
fo.config

{
    "allow_legacy_orchestrators": false,
    "batcher_static_size": 100,
    "batcher_target_latency": 0.2,
    "batcher_target_size_bytes": 1048576,
    "database_admin": true,
    "database_compressor": null,
    "database_dir": "/Users/saumyagoyal/JupyterNotebook/MLCon/MLCon_25S/AIOpsKit/MLCon/1_FiftyOne/fiftyone_root/database",
    "database_name": "fiftyone",
    "database_uri": null,
    "database_validation": true,
    "dataset_zoo_dir": "/Users/saumyagoyal/JupyterNotebook/MLCon/MLCon_25S/AIOpsKit/MLCon/1_FiftyOne/fiftyone_root/zoo/datasets",
    "dataset_zoo_manifest_paths": null,
    "default_app_address": "localhost",
    "default_app_port": 5151,
    "default_batch_size": null,
    "default_batcher": "latency",
    "default_dataset_dir": "/Users/saumyagoyal/JupyterNotebook/MLCon/MLCon_25S/AIOpsKit/MLCon/1_FiftyOne/fiftyone_root/default",
    "default_image_ext": ".jpg",
    "default_ml_backend": "torch",
    "default_parallelization_method": null,
    "default_process_pool

In [4]:
fo.annotation_config

{
    "default_backend": "labelstudio",
    "backends": {
        "cvat": {
            "config_cls": "fiftyone.utils.cvat.CVATBackendConfig",
            "url": "https://app.cvat.ai"
        },
        "labelbox": {
            "config_cls": "fiftyone.utils.labelbox.LabelboxBackendConfig",
            "url": "https://labelbox.com"
        },
        "labelstudio": {
            "url": "http://localhost:8080/",
            "api_key": "2fdcccfe63c942875fa20dfa44caccf1aff0e483",
            "config_cls": "fiftyone.utils.labelstudio.LabelStudioBackendConfig"
        }
    }
}

In [6]:
fo.list_datasets()

You are running the oldest supported major version of MongoDB. Please refer to https://deprecation.voxel51.com for deprecation notices. You can suppress this exception by setting your `database_validation` config parameter to `False`. See https://docs.voxel51.com/user_guide/config.html#configuring-a-mongodb-connection for more information


['quickstart']

In [7]:
dataset = foz.load_zoo_dataset(
    "quickstart")

Dataset already downloaded
Loading existing dataset 'quickstart'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use


In [8]:
dataset.list_saved_views()

['dogs_view', 'cluster_2_view']

In [9]:
dogs_view = dataset.load_saved_view("dogs_view")

In [10]:
annotion_key = "labelstudio_fifityone_dogs_annotations_v1"

In [11]:
label_schema = {
    "ls_groundtruth": {
        "type": "detections",
        "classes": ["dog", "cat", "unknown", "horse", "cow", "sheep", "car", "ship"],
    }
}

In [13]:
# send the data to LS for annotation

dogs_view.annotate(
    anno_key = annotion_key,
    label_schema = label_schema,
    project_name = "fiftyone_ls_integration",
)

The backend 'labelstudio' does not support attributes. Provided attributes will be ignored.


/Users/saumyagoyal/JupyterNotebook/MLCon/MLCon_25S/AIOpsKit/MLCon/.venv/lib/python3.12/site-packages/label_studio_sdk/_extensions/label_studio_tools/core/label_config.py:137: SyntaxWarning: invalid escape sequence '\$'
  expression = "^\$[A-Za-z_]+$"
/Users/saumyagoyal/JupyterNotebook/MLCon/MLCon_25S/AIOpsKit/MLCon/.venv/lib/python3.12/site-packages/label_studio_sdk/files/client.py:116: SyntaxWarning: invalid escape sequence '\ '
  """
/Users/saumyagoyal/JupyterNotebook/MLCon/MLCon_25S/AIOpsKit/MLCon/.venv/lib/python3.12/site-packages/label_studio_sdk/files/client.py:442: SyntaxWarning: invalid escape sequence '\ '
  """
/Users/saumyagoyal/JupyterNotebook/MLCon/MLCon_25S/AIOpsKit/MLCon/.venv/lib/python3.12/site-packages/label_studio_sdk/projects/client.py:520: SyntaxWarning: invalid escape sequence '\.'
  """
/Users/saumyagoyal/JupyterNotebook/MLCon/MLCon_25S/AIOpsKit/MLCon/.venv/lib/python3.12/site-packages/label_studio_sdk/projects/client.py:1235: SyntaxWarning: invalid escape sequen

Uploading media to Label Studio...
Computing metadata...
 100% |███████████████████| 10/10 [52.7ms elapsed, 0s remaining, 189.7 samples/s] 
Upload complete


In [14]:
# look at the annotations
dataset.list_annotation_runs()

['labelstudio_fifityone_dogs_annotations_v1']

In [15]:
# load the annotations back into FiftyOne

dataset.load_annotations(annotion_key)

Download complete
Loading labels for field 'ls_groundtruth'...
 100% |███████████████████| 10/10 [13.9ms elapsed, 0s remaining, 718.1 samples/s] 


In [22]:
from fiftyone import ViewField as F

dogs_annotated_view = (
    dataset
    .filter_labels("ground_truth", F("label") == "dog")
)
dataset.save_view("dogs_annotated_view_v3", dogs_annotated_view)